In [ ]:
import os
os.environ['GRAPHISTRY_USERNAME'] = input("Please enter your graphistry username")
os.environ['GRAPHISTRY_PASSWORD'] = input("Please enter your graphistry password")

In [ ]:
from cognee.modules.users.methods import get_default_user

from cognee.modules.data.methods import get_datasets
from cognee.modules.data.methods.get_dataset_data import get_dataset_data
from cognee.modules.data.models import Data

from cognee.modules.pipelines.tasks.Task import Task
from cognee.tasks.documents import classify_documents, check_permissions_on_documents, extract_chunks_from_documents
from cognee.tasks.graph import extract_graph_from_code
from cognee.tasks.storage import add_data_points
from cognee.shared.SourceCodeGraph import SourceCodeGraph

from cognee.modules.pipelines import run_tasks

from cognee.shared.utils import render_graph


In [ ]:
user = await get_default_user()

In [ ]:
existing_datasets = await get_datasets(user.id)

datasets = {}
for dataset in existing_datasets:
    dataset_name = dataset.name.replace(".", "_").replace(" ", "_")
    data_documents: list[Data] = await get_dataset_data(dataset_id = dataset.id)
    datasets[dataset_name] = data_documents
print(datasets.keys())

In [ ]:
tasks = [
    Task(classify_documents),
    Task(check_permissions_on_documents, user = user, permissions = ["write"]),
    Task(extract_chunks_from_documents), # Extract text chunks based on the document type.
    Task(add_data_points, task_config = { "batch_size": 10 }),
    Task(extract_graph_from_code, graph_model = SourceCodeGraph, task_config = { "batch_size": 10 }), # Generate knowledge graphs from the document chunks.
]

In [ ]:
async def run_codegraph_pipeline(tasks, data_documents):
    pipeline = run_tasks(tasks, data_documents, "code_graph_pipeline")
    results = []
    async for result in pipeline:
        results.append(result)
    return(results)

In [ ]:
results = await run_codegraph_pipeline(tasks, datasets["main_dataset"])

In [ ]:
await render_graph(None, include_nodes = True, include_labels = True)